In [1]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "Форма_Договор_коммерческой_концессии_франчайзинга_образе_1.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

9


In [2]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

КонсультантПлюс | Готовое решение | Актуально на 17.03.2025
Образец договора коммерческой концессии
(франчайзинга)
Применимые нормы: п. 1 ст. 1027, п. 1 ст. 1028 ГК РФ
Образец подготовлен на примере с

{'producer': 'КонсультантПлюс', 'creator': 'Версия 4024.00.50© 1992 - 2025 КонсультантПлюс Сборка 735869', 'creationdate': 'D:20250317134905', 'title': 'Форма: Договор коммерческой концессии (франчайзинга) (образец заполнения)\n(КонсультантПлюс, 2025)', 'source': 'Форма_Договор_коммерческой_концессии_франчайзинга_образе_1.pdf', 'total_pages': 9, 'page': 0, 'page_label': '1'}


In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

28

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

e:\dev\franchising-risk\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 768

[0.061135388910770416, -0.001494561554864049, 0.008928265422582626, -0.03994758799672127, -0.02074180170893669, -0.02940932661294937, 0.02348395623266697, -0.0048781209625303745, -0.0025344397872686386, 0.037024907767772675]


In [6]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [7]:
ids = vector_store.add_documents(documents=all_splits)

In [11]:
results = vector_store.similarity_search(
    "Условия досрочного отказа от договора"
)

print(results[0])

page_content='Примечание:
Если вы не укажете срок действия договора, он будет действовать 5 лет ( п. 4 ст. 1027 , п. 4 ст.
1235 ГК РФ).
3. Права и обязанности сторон
3.1. Правообладатель передает Пользователю ноу-хау - сведения о способах ведения
профессиональной деятельности ресторана "Скатерть-самобранка" путем отправки комплекта
документов курьером на адрес Пользователя в течение 2 рабочих дней с момента подписания
договора.
3.2. Правообладатель обязуется оказывать пользователю техническое и консультативное
содействие, в частности давать устные и письменные консультации:
• по вопросам ремонта помещения для ресторана, в том числе в связи с необходимостью
соблюдения санитарных норм;
• оформлению разрешительной документации;
• поставщикам оборудования;
• дизайну помещений ресторана;
• поставщикам продуктов;
• IT-поддержке, поставщикам готовых технических решений;
• маркетинговой поддержке, возможностям по увеличению выручки, посещаемости;
• иным вопросам в связи с деятельностью рестора

In [ ]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)


retriever.batch(
    [
        "Условия досрочного отказа от договора",
    ],
)[0][0].page_content

'Примечание:\nЕсли вы не укажете срок действия договора, он будет действовать 5 лет ( п. 4 ст. 1027 , п. 4 ст.\n1235 ГК РФ).\n3. Права и обязанности сторон\n3.1. Правообладатель передает Пользователю ноу-хау - сведения о способах ведения\nпрофессиональной деятельности ресторана "Скатерть-самобранка" путем отправки комплекта\nдокументов курьером на адрес Пользователя в течение 2 рабочих дней с момента подписания\nдоговора.\n3.2. Правообладатель обязуется оказывать пользователю техническое и консультативное\nсодействие, в частности давать устные и письменные консультации:\n• по вопросам ремонта помещения для ресторана, в том числе в связи с необходимостью\nсоблюдения санитарных норм;\n• оформлению разрешительной документации;\n• поставщикам оборудования;\n• дизайну помещений ресторана;\n• поставщикам продуктов;\n• IT-поддержке, поставщикам готовых технических решений;\n• маркетинговой поддержке, возможностям по увеличению выручки, посещаемости;\n• иным вопросам в связи с деятельностью ре